### Notebook to create BLOCK-T74 for LUT in Azimuth

Created on: 2024-10-09

Author: Guillem Megias

In [2]:
from lsst.ts.observing import ObservingBlock, ObservingScript 
from lsst.ts.aos.analysis import build_configuration_schema
import os

In [1]:
current_path = os.getcwd()
block_number = 'T74'
program = "BLOCK-T74"
reason = "SITCOM-1490"
note = "LUT_azimuth"
constraints = []

### Define configuration schema

In [3]:
# Define the configurable properties that we will use in the configuration schema
properties = {
    "run_filter": {
        "description": "ComCam filter to use.",
        "type": "string",
        "default": "r_03"
    },
    "run_elevation": {
        "description": "Elevation to use for the elevation sweep.",
        "type": "number",
        "default": 80
    }
}

# Build the configuration schema for BLOCK-404
configuration_schema = build_configuration_schema(block_number, properties)
print(configuration_schema)

$schema: http://json-schema.org/draft-07/schema#
title: BLOCK-T74 configuration
description: Configuration for BLOCK-T74.
type: object
properties:
  run_filter:
    description: ComCam filter to use.
    type: string
    default: "r_03"
  run_elevation:
    description: Elevation to use for the elevation sweep.
    type: number
    default: 80



### Define scripts and block

In [4]:
comcam_triplets_script = ObservingScript(
    name="maintel/take_triplet_comcam.py",
    standard=True,
    parameters= dict(
        filter="$run_filter",
        program="$program",
        reason=reason,
        note=note,
    )
)

stop_tracking_script = ObservingScript(
    name="maintel/stop_tracking.py",
    standard=True,
    parameters = dict()
)

In [6]:
azimuth_positions = [-180, -150, -120, -90, -60, -30, 0, 30, 60, 90, 120, 150, 180]
scripts = []

for azimuth in azimuth_positions:
    track_target_script = ObservingScript(
        name="maintel/track_target.py",
        standard=True,
        parameters = dict(
            track_azel = dict(
                az = azimuth,
                el = '$run_elevation'
            )
        )
    )

    scripts.append(track_target_script)
    scripts.append(comcam_triplets_script)
scripts.append(stop_tracking_script)

In [9]:
block = ObservingBlock(
    name = program,
    program = program,
    configuration_schema=configuration_schema,
    scripts = scripts,
)

### Save configurable block

In [10]:
block.model_dump_json(indent=2)

output_file_path = f'{current_path}/aos/ts_config_ocs/Scheduler/observing_blocks_maintel/AOS/LUTs/{program}.json'

with open(output_file_path, 'w') as file:
    file.write(block.model_dump_json(indent=2))